In [41]:
import torch
import torchvision
import torchvision.transforms as transforms
import tarfile
import pandas as pd
import os
import re
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from io import StringIO
from sklearn.decomposition import PCA

In [2]:
class DatasetClass(Dataset):
    
    def __init__(self, folder, filename, label_dict):
        
        self.data = []
        self.filename = filename
        tar = tarfile.open(folder + '\\' + filename)
        for file in tar.getmembers():
            f = tar.extractfile(file)
            if f != None:
                content = pd.read_csv(StringIO(f.read().decode()), sep=' ', header=None).values.ravel()
                self.data.append(content)
            
        self.y = torch.tensor(label_dict[self.filename[:-7]], dtype=torch.long)
    
    def __getitem__(self, idx):     
        
        return torch.tensor(self.data[idx], dtype=torch.float), self.y
      
    def __len__(self):
        
        return len(self.data)

In [15]:
def train_test_loader(directory, label_dict, train_fraction=0.8, num_workers=2, batch_size=32):

    all_files = list(filter(lambda x: x.endswith('.tar.gz'), os.listdir(directory)))
    files = [file for file in all_files if file[:-7] in label_dict.keys()]
    
    datasets = list(map(lambda x : DatasetClass(directory, x, label_dict), files))
    dataset = ConcatDataset(datasets)
    N = dataset.cumulative_sizes[-1]
    
    train_size = int(N*train_fraction)
    test_size = N - train_size

    train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

    trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    
    return trainloader, testloader

In [4]:
label_dict = {'tallbuilding': 0, 'opencountry':1, 'mountain': 2, 'highway': 3, 'coast': 4}
trainloader, testloader = train_test_loader('Data_Set_1(Colored_Images)', label_dict, train_fraction=0.8, num_workers=0)

In [5]:
class AutoEncoder(nn.Module):
    
    def __init__(self, n_features, h_layer_sizes):
        super(AutoEncoder, self).__init__()
        
        self.fc1 = nn.Linear(n_features, h_layer_sizes[0])
        self.fc2 = nn.Linear(h_layer_sizes[0], h_layer_sizes[1])
        self.fc3 = nn.Linear(h_layer_sizes[1], h_layer_sizes[2])
        self.out = nn.Linear(h_layer_sizes[2], n_features)
        
    def forward(self, x):
        
        x = torch.tanh(self.fc1(x)) # Hidden Layer 1 (Tanh)
        x = self.fc2(x)    # Hidden Layer 2 (Linear)
        x = torch.tanh(self.fc3(x)) # Hidden Layer 3 (Tanh)
        x = self.out(x) # Output Layer (Linear)
        
        return x
    
    def get_z(self, x):
        
        z = torch.tanh(self.fc1(x))
        z = self.fc2(z)
        
        return z

In [6]:
criterion = nn.MSELoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
ae1 = AutoEncoder(828, [500, 300, 500])
ae1 = ae1.to(device)
optimizer1 = optim.SGD(ae1.parameters(), lr=0.001, momentum=0.9)

In [8]:
old_loss = np.inf

max_epoch = 200

for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, _ = data[0].to(device), data[1].to(device)
        
        optimizer1.zero_grad()
        
        # Reconstructed Representation of X (forward)
        X_hat = ae1(X)
        
        # Calculate Loss (MSE)
        loss = criterion(X_hat, X)
        
        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer1.step()
        
        running_loss += loss.item()
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    
    
    if abs(running_loss-old_loss)/running_loss < 1e-3:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')

Epoch 1 : Loss = 82.11686182022095
Epoch 2 : Loss = 72.09533536434174
Epoch 3 : Loss = 55.275643706321716
Epoch 4 : Loss = 35.821474611759186
Epoch 5 : Loss = 21.7523130774498
Epoch 6 : Loss = 14.383622944355011
Epoch 7 : Loss = 11.11868867278099
Epoch 8 : Loss = 9.768427357077599
Epoch 9 : Loss = 9.233877211809158
Epoch 10 : Loss = 9.020542934536934
Epoch 11 : Loss = 8.934777528047562
Epoch 12 : Loss = 8.899174153804779
Epoch 13 : Loss = 8.883565053343773
Epoch 14 : Loss = 8.875580057501793
Converged
Finished Training


In [9]:
class FinalNet(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(FinalNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        #self.fc4 = nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.out = nn.Linear(hidden_sizes[2], num_classes)
    
    def forward(self, x):
        
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        #x = torch.tanh(self.fc4(x))
        x = self.out(x)
        
        return x
    
    def predict(self, X):
        
        with torch.no_grad():
            y_score = self.forward(X)
            y_pred = torch.argmax(y_score, axis=1)
            
        return y_pred
            
    
classifier = FinalNet(300, [150, 75, 50], 5)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)
classifier = classifier.to(device)

In [11]:
old_loss = np.inf

max_epoch = 500

for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, y = data[0].to(device), data[1].to(device)
        
        # extracting encoder features from AE1 to use as input to the MLFFNN
        with torch.no_grad():
            Z = ae1.get_z(X)
        
        optimizer.zero_grad()
        
        # Forward
        y_hat = classifier(Z)
        
        # Calculate Loss (Cross Entropy)
        loss = criterion(y_hat, y)
        
        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer.step()
        
        running_loss += loss.item()
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    
    if abs(running_loss-old_loss)/running_loss < 1e-4:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')

Epoch 1 : Loss = 70.59424698352814
Epoch 2 : Loss = 70.35061419010162
Epoch 3 : Loss = 70.36256313323975
Epoch 4 : Loss = 70.31681597232819
Epoch 5 : Loss = 70.32313930988312
Converged
Finished Training


In [12]:
with torch.no_grad():
    
    test_loss = 0.0
    y_test = []
    y_test_pred = []

    for data in testloader:

        X, y = data[0].to(device), data[1].to(device)
        Z = ae1.get_z(X)
        y_hat = classifier(Z)      
        test_loss += criterion(y_hat, y)
        
        y_test.extend(list(y.cpu().detach().numpy()))
        y_test_pred.extend(list(torch.argmax(y_hat, axis=1).cpu().detach().numpy()))

print('Test Loss =', test_loss.item())
pd.DataFrame(confusion_matrix(y_test, y_test_pred))

Test Loss = 17.60600471496582


,0,1,2,3,4
0,0,71,0,0,0
1,0,85,0,0,0
2,0,68,0,0,0
3,0,55,0,0,0
4,0,73,0,0,0


# PCA

In [43]:
# To fit the PCA model we load all the training points in a single batch
train, test = train_test_loader('Data_Set_1(Colored_Images)', label_dict, train_fraction=0.8, batch_size=2000, num_workers=0)

for i in train:
    print(i[0])
    temp = i[0]
    print(i[0].shape)
    break

tensor([[0.5950, 0.6597, 0.4282,  ..., 2.9600, 2.7177, 2.9838],
        [0.5868, 0.4174, 0.8629,  ..., 3.3219, 3.1987, 3.3914],
        [0.5960, 0.5398, 0.4418,  ..., 3.0622, 3.0685, 2.7189],
        ...,
        [0.6232, 0.6077, 0.6365,  ..., 3.1719, 2.7474, 2.9302],
        [0.6060, 0.0340, 0.8770,  ..., 1.9181, 2.1452, 2.0354],
        [0.5327, 0.5327, 0.4959,  ..., 2.1170, 2.1116, 2.0841]])
torch.Size([1408, 828])


In [49]:
def return_pca(temp):
    pca = PCA(n_components=0.99)
    pca.fit(temp)
    return pca

PCA_model = return_pca(temp)

In [53]:
reduced_dimension = PCA_model.transform(temp).shape[1]
pca_clf = FinalNet(reduced_dimension, [150, 75, 50], 5)

In [55]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pca_clf.parameters(), lr=0.001, momentum=0.9)
pca_clf = pca_clf.to(device)

In [57]:
old_loss = np.inf

max_epoch = 500

for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, y = data[0], data[1].to(device)
        
        # applying PCA on the data to use as input to the MLFFNN
        Z = torch.Tensor(PCA_model.transform(X)).to(device)
        
        optimizer.zero_grad()
        
        # Forward
        y_hat = pca_clf(Z)
        
        # Calculate Loss (Cross Entropy)
        loss = criterion(y_hat, y)
        
        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer.step()
        
        running_loss += loss.item()
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    
    if abs(running_loss-old_loss)/running_loss < 1e-4:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')

Epoch 1 : Loss = 70.95451056957245
Epoch 2 : Loss = 69.7358535528183
Epoch 3 : Loss = 68.4542692899704
Epoch 4 : Loss = 67.03552746772766
Epoch 5 : Loss = 65.47527801990509
Epoch 6 : Loss = 63.835047245025635
Epoch 7 : Loss = 62.26478052139282
Epoch 8 : Loss = 60.82287073135376
Epoch 9 : Loss = 59.50989067554474
Epoch 10 : Loss = 58.211318373680115
Epoch 11 : Loss = 56.8797789812088
Epoch 12 : Loss = 55.472567319869995
Epoch 13 : Loss = 53.97001278400421
Epoch 14 : Loss = 52.46879464387894
Epoch 15 : Loss = 50.976709485054016
Epoch 16 : Loss = 49.48734086751938
Epoch 17 : Loss = 48.13438868522644
Epoch 18 : Loss = 46.83879017829895
Epoch 19 : Loss = 45.588482558727264
Epoch 20 : Loss = 44.41742658615112
Epoch 21 : Loss = 43.343338429927826
Epoch 22 : Loss = 42.2998605966568
Epoch 23 : Loss = 41.31622505187988
Epoch 24 : Loss = 40.40190762281418
Epoch 25 : Loss = 39.50726789236069
Epoch 26 : Loss = 38.62200117111206
Epoch 27 : Loss = 37.84256637096405
Epoch 28 : Loss = 37.05637961626053

Epoch 223 : Loss = 0.35886241495609283
Epoch 224 : Loss = 0.3549400568008423
Epoch 225 : Loss = 0.35141371190547943
Epoch 226 : Loss = 0.3475446403026581
Epoch 227 : Loss = 0.34422467648983
Epoch 228 : Loss = 0.3403140902519226
Epoch 229 : Loss = 0.33684442937374115
Epoch 230 : Loss = 0.3338111937046051
Epoch 231 : Loss = 0.3304840326309204
Epoch 232 : Loss = 0.327055424451828
Epoch 233 : Loss = 0.3238407224416733
Epoch 234 : Loss = 0.32101453840732574
Epoch 235 : Loss = 0.3179997205734253
Epoch 236 : Loss = 0.3152262568473816
Epoch 237 : Loss = 0.31195494532585144
Epoch 238 : Loss = 0.30859918892383575
Epoch 239 : Loss = 0.3060655891895294
Epoch 240 : Loss = 0.3031214028596878
Epoch 241 : Loss = 0.30054140090942383
Epoch 242 : Loss = 0.2977461963891983
Epoch 243 : Loss = 0.29483582079410553
Epoch 244 : Loss = 0.2920280396938324
Epoch 245 : Loss = 0.28970593214035034
Epoch 246 : Loss = 0.28703485429286957
Epoch 247 : Loss = 0.284223273396492
Epoch 248 : Loss = 0.2820495516061783
Epoch 

Epoch 435 : Loss = 0.09854684770107269
Epoch 436 : Loss = 0.09817448258399963
Epoch 437 : Loss = 0.09780050814151764
Epoch 438 : Loss = 0.09739133715629578
Epoch 439 : Loss = 0.0970243364572525
Epoch 440 : Loss = 0.09668996930122375
Epoch 441 : Loss = 0.09632909297943115
Epoch 442 : Loss = 0.09596574306488037
Epoch 443 : Loss = 0.09563899040222168
Epoch 444 : Loss = 0.09527324140071869
Epoch 445 : Loss = 0.09491516649723053
Epoch 446 : Loss = 0.09455963969230652
Epoch 447 : Loss = 0.09425465762615204
Epoch 448 : Loss = 0.09386946260929108
Epoch 449 : Loss = 0.09354498982429504
Epoch 450 : Loss = 0.09320688247680664
Epoch 451 : Loss = 0.09290160238742828
Epoch 452 : Loss = 0.09254403412342072
Epoch 453 : Loss = 0.09219805896282196
Epoch 454 : Loss = 0.09190163016319275
Epoch 455 : Loss = 0.09154480695724487
Epoch 456 : Loss = 0.09124806523323059
Epoch 457 : Loss = 0.09090901911258698
Epoch 458 : Loss = 0.09062895178794861
Epoch 459 : Loss = 0.09028442203998566
Epoch 460 : Loss = 0.08995

In [58]:
with torch.no_grad():
    
    test_loss = 0.0
    y_test = []
    y_test_pred = []

    for data in testloader:

        X, y = data[0], data[1].to(device)
        Z = torch.Tensor(PCA_model.transform(X)).to(device)
        y_hat = pca_clf(Z)      
        test_loss += criterion(y_hat, y)
        
        y_test.extend(list(y.cpu().detach().numpy()))
        y_test_pred.extend(list(torch.argmax(y_hat, axis=1).cpu().detach().numpy()))

print('Test Loss =', test_loss.item())
pd.DataFrame(confusion_matrix(y_test, y_test_pred))

Test Loss = 27.340171813964844


,0,1,2,3,4
0,44,4,16,4,3
1,4,55,12,2,12
2,9,13,36,4,6
3,6,4,7,30,8
4,5,12,8,7,41
